In [1]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
load_dotenv()

True

In [4]:
DB_HOST = os.getenv('DB_HOST')
DB_USER = os.getenv('DB_USER')
DB_PASSWORD = os.getenv('DB_PASSWORD')
DB_NAME = os.getenv('DB_NAME')

In [5]:
DATABASE_URI = f'mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}'
engine = create_engine(DATABASE_URI)

In [12]:
query = '''
    SELECT 
        t.id as tour_id,
        t.name as tour_name,
        COALESCE(to_count.number_of_tourOptions, 0) as number_of_tourOptions,
        COALESCE(loc.mean_lat, 0) as mean_lat,
        COALESCE(loc.mean_long, 0) as mean_long,
        COALESCE(loc.number_of_unique_countries, 0) as number_of_unique_countries,
        COALESCE(loc.country_codes, '') as country_codes,
        COALESCE(loc.number_of_locations_visited, 0) as number_of_locations_visited,
        COALESCE(opt.full_price_min, 0) as full_price_min,
        COALESCE(opt.full_price_max, 0) as full_price_max,
        COALESCE(dep.earliest_start_date, '') as earliest_start_date,
        COALESCE(dep.last_end_date, '') as last_end_date,
        CONCAT('https://luxuryescapes.com/de/tour/', t.id) as tour_link,
        COALESCE(itin.number_of_days, 0) as number_of_days,
        COALESCE(dep.Jan, 0) as Jan,
        COALESCE(dep.Feb, 0) as Feb,
        COALESCE(dep.Mar, 0) as Mar,
        COALESCE(dep.Apr, 0) as Apr,
        COALESCE(dep.May, 0) as May,
        COALESCE(dep.Jun, 0) as Jun,
        COALESCE(dep.Jul, 0) as Jul,
        COALESCE(dep.Aug, 0) as Aug,
        COALESCE(dep.Sep, 0) as Sep,
        COALESCE(dep.Oct, 0) as Oct,
        COALESCE(dep.Nov, 0) as Nov,
        COALESCE(dep.Decc, 0) as Decc
    FROM 
        tours t
    LEFT JOIN (
        SELECT 
            tour_id,
            COUNT(*) as number_of_tourOptions
        FROM 
            tour_options
        GROUP BY 
            tour_id
    ) to_count ON t.id = to_count.tour_id
    LEFT JOIN (
        SELECT 
            tour_id,
            AVG(latitude) as mean_lat,
            AVG(longitude) as mean_long,
            COUNT(DISTINCT countryCode) as number_of_unique_countries,
            GROUP_CONCAT(DISTINCT countryCode ORDER BY countryCode ASC SEPARATOR ', ') as country_codes,
            COUNT(DISTINCT name) as number_of_locations_visited
        FROM 
            locations_visited
        GROUP BY 
            tour_id
    ) loc ON t.id = loc.tour_id
    LEFT JOIN (
        SELECT 
            tour_id,
            MIN(fullPrice) as full_price_min,
            MAX(fullPrice) as full_price_max
        FROM 
            options
        GROUP BY 
            tour_id
    ) opt ON t.id = opt.tour_id
    LEFT JOIN (
        SELECT 
            fkTourOptionId,
            MIN(startDate) as earliest_start_date,
            MAX(endDate) as last_end_date,
            MAX(CASE WHEN MONTH(startDate) = 1 OR MONTH(endDate) = 1 THEN 1 ELSE 0 END) as Jan,
            MAX(CASE WHEN MONTH(startDate) = 2 OR MONTH(endDate) = 2 THEN 1 ELSE 0 END) as Feb,
            MAX(CASE WHEN MONTH(startDate) = 3 OR MONTH(endDate) = 3 THEN 1 ELSE 0 END) as Mar,
            MAX(CASE WHEN MONTH(startDate) = 4 OR MONTH(endDate) = 4 THEN 1 ELSE 0 END) as Apr,
            MAX(CASE WHEN MONTH(startDate) = 5 OR MONTH(endDate) = 5 THEN 1 ELSE 0 END) as May,
            MAX(CASE WHEN MONTH(startDate) = 6 OR MONTH(endDate) = 6 THEN 1 ELSE 0 END) as Jun,
            MAX(CASE WHEN MONTH(startDate) = 7 OR MONTH(endDate) = 7 THEN 1 ELSE 0 END) as Jul,
            MAX(CASE WHEN MONTH(startDate) = 8 OR MONTH(endDate) = 8 THEN 1 ELSE 0 END) as Aug,
            MAX(CASE WHEN MONTH(startDate) = 9 OR MONTH(endDate) = 9 THEN 1 ELSE 0 END) as Sep,
            MAX(CASE WHEN MONTH(startDate) = 10 OR MONTH(endDate) = 10 THEN 1 ELSE 0 END) as Oct,
            MAX(CASE WHEN MONTH(startDate) = 11 OR MONTH(endDate) = 11 THEN 1 ELSE 0 END) as Nov,
            MAX(CASE WHEN MONTH(startDate) = 12 OR MONTH(endDate) = 12 THEN 1 ELSE 0 END) as Decc
        FROM 
            departures
        GROUP BY 
            fkTourOptionId
    ) dep ON t.fkTourOptionId = dep.fkTourOptionId
    LEFT JOIN (
        SELECT 
            tour_id,
            MAX(startDay) + 1 AS number_of_days
        FROM 
            itineraries
        GROUP BY 
            tour_id
    ) itin ON t.id = itin.tour_id
'''

In [13]:
data = pd.read_sql(query, engine)

In [14]:
data

,tour_id,tour_name,number_of_tourOptions,mean_lat,mean_long,number_of_unique_countries,country_codes,number_of_locations_visited,full_price_min,full_price_max,earliest_start_date,last_end_date,tour_link,number_of_days,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Decc
0,tour-00bfa641-58d2-40cc-a818-0796b344dd19,Ultimate Ireland,1,53.140420,-8.136262,2,"GB, IE",17,7325.0,10615.0,2024-04-20,2025-10-08,https://luxuryescapes.com/de/tour/tour-00bfa64...,12,0,0,0,1,1,1,1,1,1,1,0,0
1,tour-01ad08bd-8b08-4d54-a46e-f515d491f9af,Country Roads of Southern Italy & Sicily,2,39.582948,14.771509,2,"IT, VA",16,4145.0,9112.0,2023-04-15,2024-10-12,https://luxuryescapes.com/de/tour/tour-01ad08b...,16,0,0,0,1,1,1,1,1,1,1,0,0
2,tour-0364e31f-a6f9-413b-8f2e-cd2e264eb3ee,Canada's Rockies,3,51.506123,-116.816479,1,CA,9,2671.0,6854.0,2024-05-06,2025-09-28,https://luxuryescapes.com/de/tour/tour-0364e31...,9,0,0,0,0,1,1,1,1,1,0,0,0
3,tour-03fb1d9a-ab44-4e04-9c7e-6589adeb6f1a,Treasures of Spain and Portugal,2,39.908875,-4.389528,2,"ES, PT",14,3685.0,5145.0,2023-04-15,2024-11-08,https://luxuryescapes.com/de/tour/tour-03fb1d9...,15,0,0,1,1,1,1,1,1,1,1,1,0
4,tour-045e09d3-ad22-4bc9-ab2b-5849b8e781d0,Country Roads of Northern Italy,2,45.883832,10.658685,1,IT,10,3380.0,4931.0,2023-10-06,2024-10-13,https://luxuryescapes.com/de/tour/tour-045e09d...,11,0,0,0,1,1,1,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,tour-fee90207-0637-44f9-a023-7cb7b4dc205f,Victoria: Daylesford & Goldfields 2024 Small-G...,1,-37.518509,144.613801,1,AU,2,1439.4,1619.4,2024-01-30,2024-12-20,https://luxuryescapes.com/de/tour/tour-fee9020...,5,1,1,1,1,0,0,0,0,1,1,1,1
413,tour-ff0cad2e-1d45-45b4-9700-fc913046a3aa,Italy Gourmet Immersion 2024 with Chianti & Ba...,1,43.049513,11.196963,1,IT,9,3599.4,5459.4,2024-04-18,2024-10-12,https://luxuryescapes.com/de/tour/tour-ff0cad2...,11,0,0,0,1,1,1,1,1,1,1,0,0
414,tour-ff22b416-40a6-4969-a00c-02879ec69e20,"Normandy, Brittany & The Loire Valley",2,48.525841,0.398087,1,FR,12,2985.0,5075.0,2023-04-15,2024-10-19,https://luxuryescapes.com/de/tour/tour-ff22b41...,9,0,0,1,1,1,1,1,1,1,1,0,0
415,tour-ff534599-5d37-473a-9d26-243f3e949c59,Spain & Portugal in Style,1,38.988301,-4.320285,2,"ES, PT",9,6595.0,10483.0,2023-10-06,2025-10-15,https://luxuryescapes.com/de/tour/tour-ff53459...,14,0,0,0,0,1,1,1,1,1,1,0,0


In [15]:
directory = "../../data/processed"
file_path = os.path.join(directory, f"analysis.csv")
data.to_csv(file_path, index=False)

In [16]:
data.isna().sum()

tour_id                        0
tour_name                      0
number_of_tourOptions          0
mean_lat                       0
mean_long                      0
number_of_unique_countries     0
country_codes                  0
number_of_locations_visited    0
full_price_min                 0
full_price_max                 0
earliest_start_date            0
last_end_date                  0
tour_link                      0
number_of_days                 0
Jan                            0
Feb                            0
Mar                            0
Apr                            0
May                            0
Jun                            0
Jul                            0
Aug                            0
Sep                            0
Oct                            0
Nov                            0
Decc                           0
dtype: int64